# Notebook for scraping remote data science jobs off indeed.com

In [12]:
import requests
import os
import re
import time
import pandas as pd
from bs4 import BeautifulSoup

parser = 'html5lib'
tools_to_find = ['python', ' R ', 'SQL', 'D3.js', 'matlab', 'excel', 'SAS', 'BigML', 'Apache Spark', 'Jupyter', 'Tableau', 'Scikit', 'Tensorflow', 'Weka', 'Matplotlib', 'ggplot', 'nltk', 'Azure', 'BI'] # change BI next run
big_table = []

In [4]:
""" initial cell to get total number of job postings from html """

URL = "https://www.indeed.com/jobs?q=data%20science&l=Remote"
page = requests.get(URL)
soup = BeautifulSoup(page.content, parser)
res = soup.find(id="searchCountPages")
# res.text = '\n                    Page 1 of 3,337 jobs', we only need the numbers, remove everything else with regex
num_re_keep = "[^0-9]"
res_nums_string = re.sub(num_re_keep, "", res.text)
# python slice notation: list[<start>:<stop>:<step>]
# we want to get rid of the 1
total_num_jobs_str = res_nums_string[1:]
total_num_jobs_int = int(total_num_jobs_str)


AttributeError: 'NoneType' object has no attribute 'text'

In [15]:
""" cell to create the list of all the extensions for each url"""

job_num_list = []
job_step = 10
for i in range(0, total_num_jobs_int, job_step):
    str2num = str(i)
    job_num_list.append(str2num)
print(job_num_list)

['0', '10', '20', '30', '40', '50', '60', '70', '80', '90', '100', '110', '120', '130', '140', '150', '160', '170', '180', '190', '200', '210', '220', '230', '240', '250', '260', '270', '280', '290', '300', '310', '320', '330', '340', '350', '360', '370', '380', '390', '400', '410', '420', '430', '440', '450', '460', '470', '480', '490', '500', '510', '520', '530', '540', '550', '560', '570', '580', '590', '600', '610', '620', '630', '640', '650', '660', '670', '680', '690', '700', '710', '720', '730', '740', '750', '760', '770', '780', '790', '800', '810', '820', '830', '840', '850', '860', '870', '880', '890', '900', '910', '920', '930', '940', '950', '960', '970', '980', '990', '1000', '1010', '1020', '1030', '1040', '1050', '1060', '1070', '1080', '1090', '1100', '1110', '1120', '1130', '1140', '1150', '1160', '1170', '1180', '1190', '1200', '1210', '1220', '1230', '1240', '1250', '1260', '1270', '1280', '1290', '1300', '1310', '1320', '1330', '1340', '1350', '1360', '1370', '1380'

In [23]:
job_num_list.reverse()
job_num_list_rev = job_num_list
job_num_list.reverse()

print(job_num_list_rev)

['3330', '3320', '3310', '3300', '3290', '3280', '3270', '3260', '3250', '3240', '3230', '3220', '3210', '3200', '3190', '3180', '3170', '3160', '3150', '3140', '3130', '3120', '3110', '3100', '3090', '3080', '3070', '3060', '3050', '3040', '3030', '3020', '3010', '3000', '2990', '2980', '2970', '2960', '2950', '2940', '2930', '2920', '2910', '2900', '2890', '2880', '2870', '2860', '2850', '2840', '2830', '2820', '2810', '2800', '2790', '2780', '2770', '2760', '2750', '2740', '2730', '2720', '2710', '2700', '2690', '2680', '2670', '2660', '2650', '2640', '2630', '2620', '2610', '2600', '2590', '2580', '2570', '2560', '2550', '2540', '2530', '2520', '2510', '2500', '2490', '2480', '2470', '2460', '2450', '2440', '2430', '2420', '2410', '2400', '2390', '2380', '2370', '2360', '2350', '2340', '2330', '2320', '2310', '2300', '2290', '2280', '2270', '2260', '2250', '2240', '2230', '2220', '2210', '2200', '2190', '2180', '2170', '2160', '2150', '2140', '2130', '2120', '2110', '2100', '2090',

In [16]:
""" cell to get information from each job description"""
def find_tools(desc):
    tools_in_desc = []
    for tool in tools_to_find:
        if tool.lower() in desc.lower():
            tools_in_desc.append(tool)
    return tools_in_desc

In [17]:
""" cell to find all the job ids to get to the actual job descriptions on each page """

def scrape_jobs(soup, URL):
    # get the job id and company name from the cards on the page
    for card in soup.find_all('div', class_=["jobsearch-SerpJobCard"]):
        title_link = card.find('a')
        # get job name
        job_name_str = title_link.get('title')
        job_link = title_link.get('href')
        coy_name = card.find('span', class_=["company"])
        # get company name
        coy_name_str = coy_name.text
        job_link = title_link.get('href')
        # need to extract the necessary info from the href.
        """ parts:
        https://www.indeed.com/viewjob?
        jk=0bd5cc9c5ecab98a
        &
        from=serp
        &
        vjs=3   
        """
        part1 = "https://www.indeed.com/viewjob?"
        and_symb = "&"
        part2 = "from=serp"
        # get jk tag
        jk_tag_start_idx = (job_link.find("clk?") + 4)
        jk_tag_end_idx = job_link.find("&")
        jk_tag = job_link[jk_tag_start_idx: jk_tag_end_idx]
        #print(jk_tag)
        # get end tag
        vjs_tag = job_link[job_link.find("vjs"):]
        #print(vjs_tag)
        full_link = part1 + jk_tag + and_symb + part2 + and_symb + vjs_tag
        print(full_link)
        page = requests.get(full_link)
        soup = BeautifulSoup(page.content, parser)
        desc = soup.find('div', id=["jobDescriptionText"])
        if (desc == None):
            continue
        tools_list = find_tools(desc.text)
        #print(tools_list)
        # make a string to write to csv
        comma = ","
        tools_list_str = comma.join(tools_list)
        out_str = job_name_str + comma + coy_name_str + comma + tools_list_str
        print(out_str)
        big_table.append(out_str)

In [26]:
""" cell to loop through each page to extract information """

URL_base = "https://www.indeed.com/jobs?q=data+science&l=Remote&start="
URL_extn = ""
for job_num in job_num_list_rev:
    URL_extn = job_num
    #print(URL_base+URL_extn)
    URL = URL_base+URL_extn
    page = requests.get(URL)
    print(URL)
    soup = BeautifulSoup(page.content, parser)
    print(soup)
    scrape_jobs(soup, URL)

https://www.indeed.com/jobs?q=data+science&l=Remote&start=3330
<html><head>
<title>hCaptcha solve page</title>
<script async="" defer="" src="https://www.hcaptcha.com/1/api.js"></script>
</head>
<body>
<form action="/jobs?q=data+science&amp;l=Remote&amp;start=3330" method="POST">
<div class="h-captcha" data-sitekey="eb27f525-f936-43b4-91e2-95a426d4a8bd"></div>
<br/>
<input type="submit" value="Submit"/>
</form>


</body></html>
https://www.indeed.com/jobs?q=data+science&l=Remote&start=3320
<html><head>
<title>hCaptcha solve page</title>
<script async="" defer="" src="https://www.hcaptcha.com/1/api.js"></script>
</head>
<body>
<form action="/jobs?q=data+science&amp;l=Remote&amp;start=3320" method="POST">
<div class="h-captcha" data-sitekey="eb27f525-f936-43b4-91e2-95a426d4a8bd"></div>
<br/>
<input type="submit" value="Submit"/>
</form>


</body></html>
https://www.indeed.com/jobs?q=data+science&l=Remote&start=3310
<html><head>
<title>hCaptcha solve page</title>
<script async="" defer=""

KeyboardInterrupt: 

In [11]:
strTest = "hello hello"
strTest = strTest.replace("e", "u", 1)
print(strTest)

hullo hello


In [18]:
strtest = "job_7ce52af9ada1a400"
print(strtest[4:])

7ce52af9ada1a400


In [121]:
string_f = "hello world"
sub_str = "EL"
if sub_str.lower() in string_f.lower():
    print("yes")
else:
    print("no")

yes


In [19]:
big_table

['data scientist -Remote,\n\nStarbucks,python,SQL,BI',
 'Part-Time Student-Smart Connected Factory/Data and Analytics-Remote Availability,\n\nJohn Deere,BI',
 'Data Scientist, Podcasts,\n\nSpotify,python,SQL,excel,BI',
 'Data Scientist,\nIntelliGen Technologies,BI',
 'Part Time Data Science Instructor,\n\nKaplan,python,excel,BI',
 'Data Scientist (Part-Time, Remote),\n\nKBR,BI',
 'Data Scientist (Remote),\n\nDeloitte,python,SQL,Tableau,BI',
 'Data Scientist - Entry Level,\nNumerdox,python, R ,BI',
 'Senior Lecturer (Online Adjunct), MS Data Science, SBE,\n\nUniversity of Wisconsin- Superior,excel,BI',
 'Data Scientist, Music Analytics,\n\nSpotify,python,SQL,Tableau,BI',
 'Healthcare Data Analyst,\nClarify Health Solutions.,SQL,BI',
 'Data Scientist - Analytics,\nAcorn Analytics,python,excel,Jupyter,Matplotlib,BI',
 'Data Engineer, Analytics (Generalist),\n\nFacebook,python,SQL,BI',
 'Data Analytics Associate (SAS),\nClarify Health Solutions.,SQL,SAS,BI',
 'Data Scientist - Analytics,\n

In [6]:
cur_table = ['data scientist -Remote,\n\nStarbucks,python,SQL,BI',
 'Part-Time Student-Smart Connected Factory/Data and Analytics-Remote Availability,\n\nJohn Deere,BI',
 'Data Scientist, Podcasts,\n\nSpotify,python,SQL,excel,BI',
 'Data Scientist,\nIntelliGen Technologies,BI',
 'Part Time Data Science Instructor,\n\nKaplan,python,excel,BI',
 'Data Scientist (Part-Time, Remote),\n\nKBR,BI',
 'Data Scientist (Remote),\n\nDeloitte,python,SQL,Tableau,BI',
 'Data Scientist - Entry Level,\nNumerdox,python, R ,BI',
 'Senior Lecturer (Online Adjunct), MS Data Science, SBE,\n\nUniversity of Wisconsin- Superior,excel,BI',
 'Data Scientist, Music Analytics,\n\nSpotify,python,SQL,Tableau,BI',
 'Healthcare Data Analyst,\nClarify Health Solutions.,SQL,BI',
 'Data Scientist - Analytics,\nAcorn Analytics,python,excel,Jupyter,Matplotlib,BI',
 'Data Engineer, Analytics (Generalist),\n\nFacebook,python,SQL,BI',
 'Data Analytics Associate (SAS),\nClarify Health Solutions.,SQL,SAS,BI',
 'Data Scientist - Analytics,\nAcorn Analytics,python,excel,Jupyter,Matplotlib,BI',
 'DATA ANALYST,\n\nUniversity of Washington,python, R ,SAS,BI',
 'Data Scientist,\n\nKaplan,python,excel,BI',
 'Part Time Data Science Instructor,\n\nKaplan,python,excel,BI',
 'Healthcare Data Analyst,\nAmino,python,SQL,BI',
 'Director I, Data Science,\n\nLiberty Mutual Insurance,SQL,excel,SAS,BI',
 'Data Scientist, Podcasts,\n\nSpotify,python,SQL,excel,BI',
 'Director, Product Analytics and Data Science - Data Science,\n\nSlack,matlab,excel,BI',
 'Data Scientist I,\n\nWindstream Communications,python,SQL,Jupyter,Tableau,BI',
 'Data Engineer, Analytics (Generalist),\n\nFacebook,python,SQL,BI',
 'Analyst I, Data Science,\n\nLiberty Mutual Insurance,python,SQL,SAS,BI',
 'Sr. Data Science Assessment Developer,\n\nAlteryx, Inc.,BI',
 'Early career data scientist / analyst / statistical geneticist,\nData Tecnica,python,excel,BI',
 'Data Scientist,\nAvantStay,python,SQL,SAS,BI',
 'Deep Learning Data Scientist,\nLinus Health,python,BI',
 'Data Science Manager, Analytics,\n\nFacebook,SQL,matlab,SAS,BI',
 'Data Modeler,\n\nCognizant Technology Solutions,excel,BI',
 'Data Scientist, AI / NLP,\nHappify Health,python,Scikit,Tensorflow,BI',
 'Data Engineering Co-op,\n\nKeurig Dr Pepper,python,SQL,excel,Tableau,Azure,BI',
 'Online Adjunct Faculty - MBA Data Analytics,\n\nHerzing University,excel,BI',
 'Data Scientist- Remote,\n\nDivisions Maintenance Group,python,SQL,Jupyter,Tensorflow,Azure,BI',
 'Senior Data Scientist- Machine Learning Scientist,\nMedibio,python,excel,Scikit,Tensorflow,BI',
 'Data Scientist 2,\n\nPRA Health Sciences,python,excel,BI',
 'Data Analyst,\n\nGovernmentCIO LLC,python,SAS,BI',
 'Data Analyst, Finance,\n\nkraken,python,SQL,Tableau,BI',
 'Engineering Director, Data & Data Science,\nBestow,excel,SAS,BI',
 'Data Science Manager, Instagram Graph,\n\nFacebook,python,SQL,matlab,SAS,BI',
 'Data Scientist, Analytics - Actor Integrity on Marketplace,\n\nFacebook,python,SQL,BI',
 'Data Scientist-1,\n\nMattress Firm,BI',
 'Data Analyst,\n\nIBM,python,SQL,excel,SAS,BI',
 'Data Analyst,\n\nBabylist,python,SQL,excel,Jupyter,BI',
 'Senior Data Analyst,\n\nkraken,python,SQL,Tableau,BI',
 'Data Scientist - Everyday Health,\nEveryday Health - Pregnancy & Parenting,python,SQL,BI',
 'Data Scientist,\nHarvest NYC,python,SQL,BI',
 'Senior Data Analyst,\n\nZuora,SQL,excel,BI',
 'Data Scientist, Risk and Fraud,\n\nEventbrite,python,SQL,BI',
 'Director of Data & Analytics,\n\nTeamSnap, Inc.,python,SQL,Tableau,BI',
 'Junior Business Analyst - Reporting & Data Insights,\n\nTeach For America,SQL,SAS,BI',
 'Data Scientist,\n\nFlorida Blue,python, R ,SQL,SAS,BI',
 'Data Science Manager - Marketplaces (Remote),\n\nYelp,BI',
 'Data Scientist,\nAlpha Net Consulting,python,BI',
 'Manager - Data Analytics,\nTDS,SQL,excel,SAS,Tableau,BI',
 'Sr. Data Analyst (remote),\n\nAllWays Health Partners,SQL,excel,BI',
 'Staff Data Scientist (Remote, Americas),\n\nShopify,python,Tableau,Matplotlib,BI',
 'Data Scientist-1,\n\nMattress Firm,BI',
 'Data Scientist,\nQventus,python,SQL,excel,Scikit,BI',
 'Site Payment Analyst (remote),\n\nIQVIA,BI',
 'Senior Data Analyst,\n\nMighty,python,SQL,excel,Tableau,BI',
 'Data Engineer,\nRhino New York,python,SQL,BI',
 'Data Engineering Co-op,\n\nKeurig Dr Pepper,python,SQL,excel,Tableau,Azure,BI',
 'Senior Data Scientist,\n\nNVIDIA,python,excel,Tensorflow,BI',
 'Data Scientist, Advertising Business Insights,\n\nSpotify,python,SQL,Tableau,BI',
 'Data Scientist,\n\nInspire,python,SQL,excel,Jupyter,Scikit,nltk,BI',
 'Data Science – Health Care Client Success Lead,\nCowen, Inc,BI',
 'Sr Manager, Data Science,\n\nAmazon Web Services, Inc.,excel,BI',
 'Data Scientist,\n\nRealogy Holdings Corp,python,BI',
 'Lead Data Scientist,\nAtomic,SQL,Tensorflow,BI',
 'Data Scientist - Shops Seller Ecosystem,\n\nFacebook,python, R ,SQL,BI',
 'Senior Data Analyst-Marketing,\n\nKaiser Permanente,',
 'Data Scientist, NLP Text Mining,\n\nDun & Bradstreet,python,SQL,BI',
 'Data Scientist,\nAlpha Net Consulting,python,BI',
 'Senior Analyst Behavioral Science & Analytics,\n\nFlorida Blue,python, R ,SQL,excel,BI',
 'Chief Technology Officer,\n\nLovepop,excel,BI',
 'Financial Intelligence Unit Analyst, Novi,\n\nFacebook,SQL,excel,BI',
 'Data Scientist (NLP),\n\nCM Group LLC,python,SQL,excel,Tensorflow,BI',
 'Principal Data Scientist, Pricing R&D,\n\nRoot Insurance Company,SQL,BI',
 'Remote Safety Data Scientist I,\nRaland Compliance Partners,BI',
 'Data Science Manager (Remote, Americas),\n\nShopify,python,Tableau,Matplotlib,BI',
 'RESEARCH ASSOCIATE FOR ECONOMIC STUDIES,\n\nCato Institute,excel,BI',
 'Data Analyst,\n\nIBM,python,SQL,excel,SAS,BI',
 'Risk Analyst,\nFast AF,python, R ,SQL,BI',
 'Data Strategist,\n\nBabylist,python,SQL,excel,Jupyter,Tableau,BI',
 'Data Engineer, Analytics,\n\nSpokeo,python,SQL,BI',
 'Data Analyst - Financial Services,\n\nNumerator,excel,Tableau,BI',
 'Senior Data Analyst,\nTogo Group,python, R ,SQL,Tableau,BI',
 'Full Stack React + Python Dev / AI Software (Remote),\n\nInvolve.ai,python,SQL,excel,SAS,BI',
 'Insights Manager, Data & Insights Tribe,\n\nSpotify,BI',
 'Director of Data & Analytics,\n\nTeamSnap, Inc.,python,SQL,Tableau,BI',
 'Senior Data Scientist (Remote Sensing) - Sustainability,\n\nNutrien Ltd.,python, R ,matlab,Jupyter,Scikit,Tensorflow,BI',
 'Senior Data Analyst, Early Childhood Development,\n\nChild Trends,excel,SAS,Tableau,ggplot,BI',
 'Senior Data Analyst,\nCuration Health,python,SQL,excel,Tableau,BI',
 'Business Intelligence Analyst, Personalization,\n\nSpotify,python,SQL,Tableau,BI',
 'Business Analyst (Remote),\n33 Mile Radius,SQL,excel,Tableau,BI',
 'Data Scientist,\nAvantStay,python,SQL,SAS,BI',
 'Senior Data Engineer,\n\nCredit Sesame,python,SQL,BI',
 'Pricing Data Analyst - Telecommute,\n\nUnitedHealth Group,python, R ,SQL,excel,SAS,BI']

In [8]:
""" cell for processing and creating the spreadsheet and analysis"""
for i in range(0, len(cur_table)):
    cur_table[i] = cur_table[i].replace("\n", "")
    print(cur_table[i])


data scientist -Remote,Starbucks,python,SQL,BI
Part-Time Student-Smart Connected Factory/Data and Analytics-Remote Availability,John Deere,BI
Data Scientist, Podcasts,Spotify,python,SQL,excel,BI
Data Scientist,IntelliGen Technologies,BI
Part Time Data Science Instructor,Kaplan,python,excel,BI
Data Scientist (Part-Time, Remote),KBR,BI
Data Scientist (Remote),Deloitte,python,SQL,Tableau,BI
Data Scientist - Entry Level,Numerdox,python, R ,BI
Senior Lecturer (Online Adjunct), MS Data Science, SBE,University of Wisconsin- Superior,excel,BI
Data Scientist, Music Analytics,Spotify,python,SQL,Tableau,BI
Healthcare Data Analyst,Clarify Health Solutions.,SQL,BI
Data Scientist - Analytics,Acorn Analytics,python,excel,Jupyter,Matplotlib,BI
Data Engineer, Analytics (Generalist),Facebook,python,SQL,BI
Data Analytics Associate (SAS),Clarify Health Solutions.,SQL,SAS,BI
Data Scientist - Analytics,Acorn Analytics,python,excel,Jupyter,Matplotlib,BI
DATA ANALYST,University of Washington,python, R ,SAS,BI

In [13]:
""" put data into csv"""

comma=","
cols = ["job_title", "company_name"] + tools_to_find
# make dictionary to get tool name to index in row
tool_dct = {}
for i in range(len(cols)):
    tool_dct[cols[i]] = i
print(tool_dct)

{'job_title': 0, 'company_name': 1, 'python': 2, ' R ': 3, 'SQL': 4, 'D3.js': 5, 'matlab': 6, 'excel': 7, 'SAS': 8, 'BigML': 9, 'Apache Spark': 10, 'Jupyter': 11, 'Tableau': 12, 'Scikit': 13, 'Tensorflow': 14, 'Weka': 15, 'Matplotlib': 16, 'ggplot': 17, 'nltk': 18, 'Azure': 19, 'BI': 20}


In [10]:
cur_table

['data scientist -Remote,Starbucks,python,SQL,BI',
 'Part-Time Student-Smart Connected Factory/Data and Analytics-Remote Availability,John Deere,BI',
 'Data Scientist, Podcasts,Spotify,python,SQL,excel,BI',
 'Data Scientist,IntelliGen Technologies,BI',
 'Part Time Data Science Instructor,Kaplan,python,excel,BI',
 'Data Scientist (Part-Time, Remote),KBR,BI',
 'Data Scientist (Remote),Deloitte,python,SQL,Tableau,BI',
 'Data Scientist - Entry Level,Numerdox,python, R ,BI',
 'Senior Lecturer (Online Adjunct), MS Data Science, SBE,University of Wisconsin- Superior,excel,BI',
 'Data Scientist, Music Analytics,Spotify,python,SQL,Tableau,BI',
 'Healthcare Data Analyst,Clarify Health Solutions.,SQL,BI',
 'Data Scientist - Analytics,Acorn Analytics,python,excel,Jupyter,Matplotlib,BI',
 'Data Engineer, Analytics (Generalist),Facebook,python,SQL,BI',
 'Data Analytics Associate (SAS),Clarify Health Solutions.,SQL,SAS,BI',
 'Data Scientist - Analytics,Acorn Analytics,python,excel,Jupyter,Matplotlib,

In [14]:
""" convert cur_table rows into new table row """
new_table = []

for i in range(len(cur_table)):
    empty_row = [0] * len(cols)
    ret_l = empty_row
    cur_list = cur_table[i].split(comma)
    print("cur_list: ")
    print(cur_list)
    # title will be first element
    ret_l[0] = cur_list[0]
    # company will be everything until first tool
    first_tool_idx = -1
    for i in range(len(cur_list)):
        if cur_list[i] in cols:
            first_tool_idx = i
            break
    if (first_tool_idx < 0):
        print("something went wrong")
        break
    ret_l[1] = ("_").join(cur_list[1:first_tool_idx])
    # encode the tools
    for i in range(len(cur_list)):
        if cur_list[i] in tool_dct:
            ret_l[tool_dct[cur_list[i]]] = 1
    print("ret_l: ")
    print(ret_l)
    print("\n")
    new_table.append(ret_l)

df = pd.DataFrame(new_table, columns=cols)
print(df)


cur_list: 
['data scientist -Remote', 'Starbucks', 'python', 'SQL', 'BI']
ret_l: 
['data scientist -Remote', 'Starbucks', 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


cur_list: 
['Part-Time Student-Smart Connected Factory/Data and Analytics-Remote Availability', 'John Deere', 'BI']
ret_l: 
['Part-Time Student-Smart Connected Factory/Data and Analytics-Remote Availability', 'John Deere', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


cur_list: 
['Data Scientist', ' Podcasts', 'Spotify', 'python', 'SQL', 'excel', 'BI']
ret_l: 
['Data Scientist', ' Podcasts_Spotify', 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


cur_list: 
['Data Scientist', 'IntelliGen Technologies', 'BI']
ret_l: 
['Data Scientist', 'IntelliGen Technologies', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


cur_list: 
['Part Time Data Science Instructor', 'Kaplan', 'python', 'excel', 'BI']
ret_l: 
['Part Time Data Science Instructor', 'Kaplan', 1, 0, 0, 0, 0, 1, 0, 0, 0, 0

In [12]:
# df to csv



<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [6]:

#print(cols)


    

,job_title,company_name,python,R,SQL,D3.js,matlab,excel,SAS,BigML,...,Jupyter,Tableau,Scikit,Tensorflow,Weka,Matplotlib,ggplot,nltk,Azure,BI


In [19]:
x = ["a"] * 10
("-").join(x)

'a-a-a-a-a-a-a-a-a-a'

In [5]:
new_table

NameError: name 'new_table' is not defined